In [528]:
# !conda update -n base -c defaults conda

# !pip install geocoder
print('geocoder has been installed.')

# !pip install geopy
print('geopy has been installed.')

# !pip install folium=0.5.0
print('folium has been installed.')

# !pip install beautifulsoup4
print('beautifulsoup4 has been installed.')

# !pip install lxml
print('lxml has been installed.')

# !pip install html5lib
print('html5lib has been installed.')

# !pip install geopandas
print('geopandas has been installed.')

geocoder has been installed.
geopy has been installed.
folium has been installed.
beautifulsoup4 has been installed.
lxml has been installed.
html5lib has been installed.
geopandas has been installed.


In [529]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib as mp
from bs4 import BeautifulSoup
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
import folium # map rendering library
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import csv
import sys
import urllib3
import lxml
import html5lib
import urllib.request
import re
from requests.exceptions import RequestException
from contextlib import closing
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')


Libraries imported.


In [530]:
# Parse the html content
# soup_Paris = BeautifulSoup(url_Paris, "lxml") (works!!)
# print(soup_Paris.prettify()) # print the parsed data of html (works!!)
# list(soup_Paris.children) (works!!)

In [531]:
# import Wikipedia URL for Paris
# Make a GET request to fetch the raw HTML content
url_Paris = requests.get('https://en.wikipedia.org/wiki/Arrondissements_of_Paris').text
# print(url_Paris)

# print(url_Paris.content) (does not work)
# print(url_Paris.status_code)   # Check the http response status code. This should print 200 (does not work)

soup_Paris = BeautifulSoup(url_Paris, "html.parser")
table_Paris = soup_Paris.find('table',{'class':'wikitable'})

# To get the text without the HTML tags, we just use .text:
print(soup_Paris.title.text)


Arrondissements of Paris - Wikipedia


In [532]:
# extracting table from https://en.wikipedia.org/wiki/Arrondissements_of_Paris
totals = table_Paris.find_all('tr')
nrows = len(totals)

# Header of the DataFrame is the first row of data
header_Paris = totals[0].text.split()
# header_Paris

In [533]:
# Creating the Table
records = []
n=1
while n < nrows :
    District = totals[n].text.split('\n')[1]
    Arrondissement = totals[n].text.split('\n')[2]
    Size = totals[n].text.split('\n')[3]
    Population = totals[n].text.split('\n')[5]
    Population_density = totals[n].text.split('\n')[6]
    records.append((District, Arrondissement, Size, Population, Population_density))
    n=n+1

df_Paris=pd.DataFrame(records, columns=['District', 'Arrondissement','Size km²', 'Population', 'Population Density'])

df_Paris.head(20)

,District,Arrondissement,Size km²,Population,Population Density
0,1st (Ie) R,Louvre,1.826 km2 (0.705 sq mi),"17,700","9,693"
1,2nd (IIe) R,Bourse,0.992 km2 (0.383 sq mi),"20,700","20,867"
2,3rd (IIIe) R,Temple,1.171 km2 (0.452 sq mi),"35,100","29,974"
3,4th (IVe) R,Hôtel-de-Ville,1.601 km2 (0.618 sq mi),"28,600","17,864"
4,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),"60,600","23,849"
5,6th (VIe) L,Luxembourg,2.154 km2 (0.832 sq mi),"45,200","20,984"
6,7th (VIIe) L,Palais-Bourbon,4.088 km2 (1.578 sq mi),"55,400","13,552"
7,8th (VIIIe) R,Élysée,3.881 km2 (1.498 sq mi),"38,700","9,972"
8,9th (IXe) R,Opéra,2.179 km2 (0.841 sq mi),"58,500","26,847"
9,10th (Xe) R,Entrepôt,2.892 km2 (1.117 sq mi),"88,800","30,705"


In [534]:
# Splitting the District column: new data frame with split value columns 
new1 = df_Paris["District"].str.split(" ", n = 2, expand = True)
  
# making separate district column from new data frame 
df_Paris["Districts"]= new1[0]

# making separate arrondissement (FR) column from new data frame 
df_Paris["Arrfr"]= new1[1]

# making separate River Bank column from new data frame 
df_Paris["RiverBank"]= new1[2]
  
# Dropping old Name columns 
df_Paris.drop(columns =['District', 'Arrfr'], inplace = True)

#df_Paris.reindex(columns="Districts","Arrondissement","RiverBank","Population","Population Density","Size km²")
# df_Paris.reset_index()
# check dataframe
df_Paris

,Arrondissement,Size km²,Population,Population Density,Districts,RiverBank
0,Louvre,1.826 km2 (0.705 sq mi),"17,700","9,693",1st,R
1,Bourse,0.992 km2 (0.383 sq mi),"20,700","20,867",2nd,R
2,Temple,1.171 km2 (0.452 sq mi),"35,100","29,974",3rd,R
3,Hôtel-de-Ville,1.601 km2 (0.618 sq mi),"28,600","17,864",4th,R
4,Panthéon,2.541 km2 (0.981 sq mi),"60,600","23,849",5th,L
5,Luxembourg,2.154 km2 (0.832 sq mi),"45,200","20,984",6th,L
6,Palais-Bourbon,4.088 km2 (1.578 sq mi),"55,400","13,552",7th,L
7,Élysée,3.881 km2 (1.498 sq mi),"38,700","9,972",8th,R
8,Opéra,2.179 km2 (0.841 sq mi),"58,500","26,847",9th,R
9,Entrepôt,2.892 km2 (1.117 sq mi),"88,800","30,705",10th,R


In [535]:
# splitting the Size column: new data frame with split value columns
new2 = df_Paris["Size km²"].str.split("(", n = 1, expand = True)
# making separate size in km² column from new data frame 
df_Paris["Size km"]= new2[0]

# making separate size in miles² column from new data frame 
df_Paris["Size miles"] = new2[1]

# Dropping old Name columns 
df_Paris.drop(columns = ["Size km²", "Size miles"], inplace = True)


In [536]:
# manually edit Population Density for 11th and 15th districts
df_Paris["Population Density"][11] = 21687
df_Paris["Population Density"][15] = 190544
df_Paris

,Arrondissement,Population,Population Density,Districts,RiverBank,Size km
0,Louvre,"17,700","9,693",1st,R,1.826 km2
1,Bourse,"20,700","20,867",2nd,R,0.992 km2
2,Temple,"35,100","29,974",3rd,R,1.171 km2
3,Hôtel-de-Ville,"28,600","17,864",4th,R,1.601 km2
4,Panthéon,"60,600","23,849",5th,L,2.541 km2
5,Luxembourg,"45,200","20,984",6th,L,2.154 km2
6,Palais-Bourbon,"55,400","13,552",7th,L,4.088 km2
7,Élysée,"38,700","9,972",8th,R,3.881 km2
8,Opéra,"58,500","26,847",9th,R,2.179 km2
9,Entrepôt,"88,800","30,705",10th,R,2.892 km2


In [537]:
# splitting the Size column: new data frame with split value columns (# not working: still showing km² after value)
new3 = df_Paris["Size km"].str.split(" ", n = 1, expand = True)
# making separate size in km² column from new data frame 
df_Paris["Size km²"]= new3[0]

# making separate size in miles² column from new data frame 
df_Paris["to drop"] = new3[1]

# Dropping old Name columns 
df_Paris.drop(columns = ["to drop", "Size km"], inplace = True)
df_Paris

,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²
0,Louvre,"17,700","9,693",1st,R,1.826 km2
1,Bourse,"20,700","20,867",2nd,R,0.992 km2
2,Temple,"35,100","29,974",3rd,R,1.171 km2
3,Hôtel-de-Ville,"28,600","17,864",4th,R,1.601 km2
4,Panthéon,"60,600","23,849",5th,L,2.541 km2
5,Luxembourg,"45,200","20,984",6th,L,2.154 km2
6,Palais-Bourbon,"55,400","13,552",7th,L,4.088 km2
7,Élysée,"38,700","9,972",8th,R,3.881 km2
8,Opéra,"58,500","26,847",9th,R,2.179 km2
9,Entrepôt,"88,800","30,705",10th,R,2.892 km2


In [538]:
# Latitude and Longitude per District
# From https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true
# unable to get it directly from the link
lat_long_Paris = pd.read_csv("arrondissements.csv", sep=';').sort_values('C_AR').reset_index()
lat_long_Paris.drop(columns = ["OBJECTID", "L_AR","N_SQ_AR","index","N_SQ_CO","Geometry"], inplace = True)

lat_long_Paris.columns = ['C_AR', 'C_ARINSEE', 'L_AROFF', 'SURFACE', 'PERIMETRE', 'Geometry', 'LONGUEUR']



In [539]:
# Splitting Geometry into separate columns to obtain latitude and longitude

# splitting the Size column: new data frame with split value columns
new4 = lat_long_Paris["Geometry"].str.split(",", n = 1, expand = True)

# making separate first name column from new data frame
lat_long_Paris["Latitude"]= new4[0]

# making separate last name column from new data frame 
lat_long_Paris["Longitude"] = new4[1]

# Dropping old Name columns 
lat_long_Paris.drop(columns = ["Geometry"], inplace = True)

#lat_long_Paris
#list(lat_long_Paris)

# convert to a Pandas dataframe
lat_long_Paris_df = pd.DataFrame(lat_long_Paris, columns= ['C_AR', 'C_ARINSEE', 'L_AROFF', 'SURFACE', 'PERIMETRE', 'LONGUEUR', 'Latitude', 'Longitude'])
lat_long_Paris_df

,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude
0,1,75101,Louvre,1.824613e+06,6054.936862,6054.680862,48.8625627018,2.33644336205
1,2,75102,Bourse,9.911537e+05,4554.104360,4553.938764,48.8682792225,2.34280254689
2,3,75103,Temple,1.170883e+06,4519.263648,4519.071982,48.86287238,2.3600009859
3,4,75104,Hôtel-de-Ville,1.600586e+06,5420.908434,5420.636779,48.8543414263,2.35762962032
4,5,75105,Panthéon,2.539375e+06,6239.195396,6238.823408,48.8444431505,2.35071460958
5,6,75106,Luxembourg,2.153096e+06,6483.686786,6483.324560,48.8491303586,2.33289799905
6,7,75107,Palais-Bourbon,4.090057e+06,8099.424883,8099.033151,48.8561744288,2.31218769148
7,8,75108,Élysée,3.880036e+06,7880.533268,7880.260661,48.8727208374,2.3125540224
8,9,75109,Opéra,2.178303e+06,6471.588290,6471.389144,48.8771635173,2.33745754348
9,10,75110,Entrepôt,2.891739e+06,6739.375055,6739.168359,48.8761300365,2.36072848785


In [540]:
# combining the two dataframes
paris_et_arrondissements_df = pd.concat([df_Paris, lat_long_Paris_df], axis=1)
lat_long_Paris_df = pd.DataFrame(lat_long_Paris, columns= ['C_AR',  'C_ARINSEE', 'Arrondissement', 'Districts', 'RiverBank', 'Latitude', 'Longitude', 'Population', 'Population Density', 'Size km²', 'PERIMETRE', 'LONGUEUR', 'SURFACE'])

paris_et_arrondissements_df


,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude
0,Louvre,"17,700","9,693",1st,R,1.826 km2,1,75101,Louvre,1.824613e+06,6054.936862,6054.680862,48.8625627018,2.33644336205
1,Bourse,"20,700","20,867",2nd,R,0.992 km2,2,75102,Bourse,9.911537e+05,4554.104360,4553.938764,48.8682792225,2.34280254689
2,Temple,"35,100","29,974",3rd,R,1.171 km2,3,75103,Temple,1.170883e+06,4519.263648,4519.071982,48.86287238,2.3600009859
3,Hôtel-de-Ville,"28,600","17,864",4th,R,1.601 km2,4,75104,Hôtel-de-Ville,1.600586e+06,5420.908434,5420.636779,48.8543414263,2.35762962032
4,Panthéon,"60,600","23,849",5th,L,2.541 km2,5,75105,Panthéon,2.539375e+06,6239.195396,6238.823408,48.8444431505,2.35071460958
5,Luxembourg,"45,200","20,984",6th,L,2.154 km2,6,75106,Luxembourg,2.153096e+06,6483.686786,6483.324560,48.8491303586,2.33289799905
6,Palais-Bourbon,"55,400","13,552",7th,L,4.088 km2,7,75107,Palais-Bourbon,4.090057e+06,8099.424883,8099.033151,48.8561744288,2.31218769148
7,Élysée,"38,700","9,972",8th,R,3.881 km2,8,75108,Élysée,3.880036e+06,7880.533268,7880.260661,48.8727208374,2.3125540224
8,Opéra,"58,500","26,847",9th,R,2.179 km2,9,75109,Opéra,2.178303e+06,6471.588290,6471.389144,48.8771635173,2.33745754348
9,Entrepôt,"88,800","30,705",10th,R,2.892 km2,10,75110,Entrepôt,2.891739e+06,6739.375055,6739.168359,48.8761300365,2.36072848785


In [541]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, name, codepostal in zip(paris_et_arrondissements_df['Latitude'], paris_et_arrondissements_df['Longitude'], paris_et_arrondissements_df['C_AR'], paris_et_arrondissements_df['Arrondissement'], paris_et_arrondissements_df['C_ARINSEE']):
        label = '{} ({}) - {}'.format(dist, name, codepostal)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)


In [542]:
# Creating map of Paris with its 20 arrondissements (districts)
latitude = 48.8566
longitude= 2.3522

# create map of Paris using latitude and longitude values above:
map_Paris = folium.Map(location=[latitude, longitude], tiles='Mapbox Bright', zoom_start=12)

df_gpd_arrondissements = gpd.read_file('arrondissements.geojson')
# df_gpd_arrondissements.head()

# map_Paris.choropleth(geo_path='arrondissements.geojson')
s = folium.GeoJson(df_gpd_arrondissements).add_to(map_Paris)

addToMap(s, 'red', map_Paris)
map_Paris

In [543]:
# we resume here

## Foursquare API

In [544]:
# FourSquare Login
CLIENT_ID = 'RU04DQE0VHYJ0KWDVSX324MAPC43P0PT0ADKYYRAADXRIE4Q' # Foursquare ID
CLIENT_SECRET = 'QXFOBOGOIZRPOCQG5WCYMDZJKT4K4G0ERKJLR1YU5IGETJYM' # my Foursquare Secret
VERSION = '20191118' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

In [545]:
# function to retrieve nearby avenues
def getNearbyVenues(names, latitudes, longitudes, radius=500, categoryIds=''):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        if (categoryIds != ''):
            url = url + '&categoryId={}'
            url = url.format(categoryIds)
    
        # make the GET request
        response = requests.get(url).json()
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category' ]
    
    return(nearby_venues)

In [546]:
# Run the function, getNearbyVenues on each neighborhood and create a new dataframe called Paris_venues
Paris_venues = getNearbyVenues(names = paris_et_arrondissements_df['C_AR'],
                                   latitudes=paris_et_arrondissements_df['Latitude'],
                                   longitudes=paris_et_arrondissements_df['Longitude']
                                    )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [547]:
print(Paris_venues.shape)

(1364, 7)


In [548]:
list(Paris_venues)

['District',
 'District Latitude',
 'District Longitude',
 'Venue',
 'Venue Latitude',
 'Venue Longitude',
 'Venue Category']

In [549]:
# Number of venues per category
venues_per_category = Paris_venues[['Venue','Venue Category']].groupby('Venue Category').count().sort_values('Venue', ascending=False)
venues_per_category

,Venue
Venue Category,
French Restaurant,176
Hotel,90
Italian Restaurant,50
Café,44
Bistro,42
Bakery,40
Plaza,36
Japanese Restaurant,36
Bar,35


In [550]:
# Number of venues per Districts
Paris_venues[['District','Venue']].groupby('District').count().sort_values('District')

,Venue
District,
1,100
2,100
3,100
4,100
5,96
6,91
7,100
8,64
9,100


In [551]:
# Number of unique categories
print('The number of unique categories is {}.'.format(len(Paris_venues['Venue Category'].unique())))

The number of unique categories is 205.


In [552]:
Paris_venues['Venue Category'].unique()

array(['Art Museum', 'Historic Site', 'Theater', 'Plaza', 'Hotel',
       'Italian Restaurant', 'Ramen Restaurant', 'Shopping Mall', 'Café',
       'Smoke Shop', 'Sculpture Garden', 'Garden', 'Gift Shop',
       'Pedestrian Plaza', 'Bar', 'Tea Room', 'Cosmetics Shop', 'Exhibit',
       'Spa', 'Furniture / Home Store', 'Japanese Restaurant', 'Bistro',
       'Bakery', 'Udon Restaurant', 'Coffee Shop', 'French Restaurant',
       'Restaurant', 'Shoe Store', 'Perfume Shop', 'Breakfast Spot',
       'Auditorium', 'Cheese Shop', 'Korean Restaurant',
       'New American Restaurant', 'Grocery Store', 'Jewelry Store',
       'Sandwich Place', 'Pizza Place', 'Cocktail Bar', 'Dessert Shop',
       'Bridge', 'Brasserie', 'Vietnamese Restaurant',
       'Chinese Restaurant', 'Wine Shop', 'Wine Bar', 'Nightclub',
       "Women's Store", 'Lebanese Restaurant', 'Tapas Restaurant',
       'Miscellaneous Shop', "Ch'ti Restaurant", 'Boutique',
       'Peruvian Restaurant', 'Beer Bar', 'Thai Restaurant'

In [553]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, venue, venueCat in zip(Paris_venues['Venue Latitude'], Paris_venues['Venue Longitude'], Paris_venues['District'], Paris_venues['Venue'], Paris_venues['Venue Category']):
        label = '{} ({}) - {}'.format(dist, venue, venueCat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [554]:
# Plot the venue
map_Paris_venues = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(Paris_venues, 'blue', map_Paris_venues)
map_Paris_venues


In [555]:
list(Paris_venues)

['District',
 'District Latitude',
 'District Longitude',
 'Venue',
 'Venue Latitude',
 'Venue Longitude',
 'Venue Category']

In [556]:
# Analyzing the top 3 venues: French Restaurant, Hotel, Italian Restaurant
df_cat_french_rest = Paris_venues.loc[Paris_venues['Venue Category'] == 'French Restaurant']
df_cat_hotel = Paris_venues.loc[Paris_venues['Venue Category'] == 'Hotel']
df_cat_italian_rest = Paris_venues.loc[Paris_venues['Venue Category'] == 'Italian Restaurant']

In [557]:
# Displaying French Restaurant
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, venue, venueCat in zip(df_cat_french_rest['Venue Latitude'], df_cat_french_rest['Venue Longitude'], df_cat_french_rest['District'], df_cat_french_rest['Venue'], df_cat_french_rest['Venue Category']):
        label = '{} ({}) - {}'.format(dist, venue, venueCat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_Paris_french_rest = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(df_cat_french_rest, 'orange', map_Paris_french_rest)
map_Paris_french_rest

In [558]:
# Top 5 districts to eat at a French Restaurant
#df_cat_italian_rest
df_cat_french_rest.groupby('District')['Venue'].count().nlargest(5)

District
7    18
9    17
4    16
8    15
5    12
Name: Venue, dtype: int64

In [559]:
# Displaying Hotels
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, venue, venueCat in zip(df_cat_hotel['Venue Latitude'], df_cat_hotel['Venue Longitude'], df_cat_hotel['District'], df_cat_hotel['Venue'], df_cat_hotel['Venue Category']):
        label = '{} ({}) - {}'.format(dist, venue, venueCat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_Paris_hotel = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(df_cat_hotel, 'yellow', map_Paris_hotel)
map_Paris_hotel

In [560]:
# Top 5 districts to go to a hotel
#df_cat_italian_rest
df_cat_hotel.groupby('District')['Venue'].count().nlargest(5)

District
7     21
9     10
1      8
8      8
17     8
Name: Venue, dtype: int64

In [561]:
# Displaying Italian Restaurant
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, dist, venue, venueCat in zip(df_cat_italian_rest['Venue Latitude'], df_cat_italian_rest['Venue Longitude'], df_cat_italian_rest['District'], df_cat_italian_rest['Venue'], df_cat_italian_rest['Venue Category']):
        label = '{} ({}) - {}'.format(dist, venue, venueCat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_Paris_italian_rest = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(df_cat_italian_rest, 'green', map_Paris_italian_rest)
map_Paris_italian_rest

In [562]:
# Top 5 districts to eat at an Italian Restaurant
#df_cat_italian_rest
df_cat_italian_rest.groupby('District')['Venue'].count().nlargest(5)

District
6     9
7     5
2     4
15    4
17    4
Name: Venue, dtype: int64

In [563]:
# Analyzing Paris Arrondissements (Districts)
# one hot encoding
Paris_onehot = pd.get_dummies(Paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Paris_onehot['District'] = Paris_venues['District'] 

# move district column to the first column
cols = list(Paris_onehot.columns.values)
cols.pop(cols.index('District'))
Paris_onehot = Paris_onehot[['District']+cols]

# rename Neighborhood for Districts so that future merge works
Paris_onehot.rename(columns = {'District': 'District'}, inplace = True)


In [564]:
Paris_onehot.head()

,District,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [565]:
Paris_onehot.shape

(1364, 206)

In [566]:
Paris_onehot.describe()

,District,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
count,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000000,1364.000

In [567]:
# Grouping rows by district and by the mean of the frequency of occurrence of each category
Paris_grouped = Paris_onehot.groupby('District').mean().reset_index()
# Paris_grouped

In [568]:
Paris_grouped.shape

(20, 206)

In [569]:
# Printing districts along with the top 5 most common venues
num_top_venues = 10

for hood in Paris_grouped['District']:
    print("----" + str(hood) + "----")
    temp = Paris_grouped[Paris_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----1----
                 venue  freq
0    French Restaurant  0.09
1                Hotel  0.08
2  Japanese Restaurant  0.07
3                 Café  0.07
4                Plaza  0.06
5          Coffee Shop  0.05
6           Art Museum  0.03
7           Restaurant  0.03
8      Udon Restaurant  0.03
9               Bakery  0.03


----2----
                 venue  freq
0    French Restaurant  0.10
1             Wine Bar  0.06
2         Cocktail Bar  0.05
3   Italian Restaurant  0.04
4  Japanese Restaurant  0.04
5               Bistro  0.04
6               Bakery  0.03
7                Hotel  0.03
8           Restaurant  0.03
9         Burger Joint  0.02


----3----
                 venue  freq
0    French Restaurant  0.08
1                 Café  0.04
2         Burger Joint  0.04
3          Coffee Shop  0.04
4               Bistro  0.04
5       Sandwich Place  0.03
6  Moroccan Restaurant  0.03
7               Bakery  0.03
8       Clothing Store  0.03
9             Wine Bar  0.03


----4--

In [570]:
list(Paris_grouped)

['District',
 'Afghan Restaurant',
 'African Restaurant',
 'American Restaurant',
 'Antique Shop',
 'Argentinian Restaurant',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auditorium',
 'Baby Store',
 'Bagel Shop',
 'Bakery',
 'Bar',
 'Basque Restaurant',
 'Bed & Breakfast',
 'Beer Bar',
 'Beer Garden',
 'Beer Store',
 'Belgian Restaurant',
 'Bike Rental / Bike Share',
 'Bistro',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Boxing Gym',
 'Brasserie',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bridge',
 'Bubble Tea Shop',
 'Burger Joint',
 'Burgundian Restaurant',
 'Bus Line',
 'Bus Station',
 'Bus Stop',
 'Butcher',
 'Café',
 'Cajun / Creole Restaurant',
 'Cambodian Restaurant',
 'Canal',
 'Candy Store',
 "Ch'ti Restaurant",
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'Comedy Club',
 'Comfort Food Restaurant',
 'Concert Hall',
 'Convenience Store',


In [571]:
# Function to define the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)    
    return row_categories_sorted.index.values[0:num_top_venues]

In [572]:
# Display the top 10 venues of each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['District'] = Paris_grouped['District']

for ind in np.arange(Paris_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,French Restaurant,Hotel,Café,Japanese Restaurant,Plaza,Coffee Shop,Restaurant,Bar,Udon Restaurant,Bakery
1,2,French Restaurant,Wine Bar,Cocktail Bar,Bistro,Japanese Restaurant,Italian Restaurant,Bakery,Restaurant,Hotel,Thai Restaurant
2,3,French Restaurant,Bistro,Burger Joint,Café,Coffee Shop,Bakery,Sandwich Place,Japanese Restaurant,Italian Restaurant,Moroccan Restaurant
3,4,French Restaurant,Ice Cream Shop,Pastry Shop,Hotel,Wine Bar,Art Gallery,Bakery,Pedestrian Plaza,Jewish Restaurant,Thai Restaurant
4,5,French Restaurant,Café,Hotel,Museum,Plaza,Bar,Science Museum,Creperie,Italian Restaurant,Bakery
5,6,French Restaurant,Italian Restaurant,Bakery,Wine Bar,Bistro,Pastry Shop,Hotel,Ice Cream Shop,Bookstore,Plaza
6,7,Hotel,French Restaurant,Café,Italian Restaurant,Plaza,History Museum,Cocktail Bar,Ice Cream Shop,Cheese Shop,Bistro
7,8,French Restaurant,Hotel,Bakery,Japanese Restaurant,Thai Restaurant,Art Gallery,Cocktail Bar,Theater,Bistro,Spa
8,9,French Restaurant,Hotel,Cocktail Bar,Bakery,Bistro,Japanese Restaurant,Wine Bar,Lounge,Pizza Place,Gourmet Shop
9,10,French Restaurant,Bistro,Hotel,Coffee Shop,Indian Restaurant,Café,Pizza Place,Japanese Restaurant,Restaurant,Italian Restaurant


In [573]:
# set number of clusters
kclusters = 5

Paris_grouped_clustering = Paris_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 4, 4, 4, 0])

In [574]:
# dataframe for clusters and top 5 venues for each district

# Clustering Districts
Paris_merged = pd.concat([Paris_grouped,paris_et_arrondissements_df], axis=1)

# add clustering labels
Paris_merged['Cluster Labels'] = kmeans.labels_

# merge Paris_grouped with Paris_data to add latitude/longitude for each neighborhood
Paris_merged = Paris_merged.join(districts_venues_sorted.set_index('District'), on='District')

Paris_merged.head(21) # check the last columns!

,District,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.030000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.010000,0.00,0.010000,0.000000,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.000000,0.010000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.

In [575]:
# visualizing different arrondissements of Paris
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, Dist, cluster in zip(Paris_merged['Latitude'], Paris_merged['Longitude'], Paris_merged['District'], Paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



In [576]:
# Cluster 1 - Most common venues: Restaurants, Hotels and Bars
cluster_1 = Paris_merged.loc[Paris_merged['Cluster Labels'] == 1, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]
cluster_1

,District,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,13,0.0,0.0,0.0,0.0,0.189655,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.086207,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.017241,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.068966,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017241

In [577]:
# Displaying 1st cluster (Cluster Labels = 0)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_1['Latitude'], cluster_1['Longitude'], cluster_1['District'], cluster_1['Cluster Labels'], cluster_1['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_1 = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(cluster_1, 'red', map_cluster_1)
map_cluster_1

In [578]:
# Cluster 2 - Most common venues: Restaurants and parks (No Hotels in Top 10 venues)
cluster_2 = Paris_merged.loc[Paris_merged['Cluster Labels'] == 2, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]
cluster_2

,District,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [579]:
# Displaying 2nd cluster (Cluster Labels = 1)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_2['Latitude'], cluster_2['Longitude'], cluster_2['District'], cluster_2['Cluster Labels'], cluster_2['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_2 = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(cluster_2, 'red', map_cluster_2)
map_cluster_2

In [580]:
# Cluster 3 - Most common venues: Nightclubs , Stations, Bars (No Hotels in Top 10 venues)
cluster_3 = Paris_merged.loc[Paris_merged['Cluster Labels'] == 3, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]
cluster_3

,District,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,16,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [581]:
# Displaying 3rd cluster (Cluster Labels = 2)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_3['Latitude'], cluster_3['Longitude'], cluster_3['District'], cluster_3['Cluster Labels'], cluster_3['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_3 = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(cluster_3, 'red', map_cluster_3)
map_cluster_3

In [582]:
# Cluster 4 - Most common venues: Coffee shops, Restaurants, Grocery stores (No Hotels in Top 10 venues)
cluster_4 = Paris_merged.loc[Paris_merged['Cluster Labels'] == 4, Paris_merged.columns[[0] + list(range(5, Paris_merged.shape[1]))]]
cluster_4

,District,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music School,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit,Arrondissement,Population,Population Density,Districts,RiverBank,Size km²,C_AR,C_ARINSEE,L_AROFF,SURFACE,PERIMETRE,LONGUEUR,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,7,0.01,0.000000,0.020000,0.00,0.0,0.0,0.0,0.0,0.0,0.010000,0.010000,0.01,0.0,0.0,0.0,0.0,0.0,0.000000,0.020000,0.0,0.0,0.0,0.0,0.010000,0.00,0.01,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.060000,0.0,0.0,0.0,0.00,0.0,0.02,0.00,0.01,0.0,0.03000,0.02,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.0,0.0,0.00,0.01,0.010000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.010000,0.0,0.180000,0.000000,0.02,0.0,0.0,0.0,0.00,0.0,0.01,0.0,0.010000,0.000000,0.000000,0.0,0.0,0.02,0.03,0.0,0.210000,0.000000,0.02,0.000000,0.0,0.0,0.000000,0.01,0.0,0.050000,0.020000,0.0,0.0,0.00,0.0,0.010000,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0

In [583]:
# Displaying 4th cluster (Cluster Labels = 3)
def addToMap(df, color, existingMap):
    for lat, lng, dist, cluster, venue1 in zip(cluster_4['Latitude'], cluster_4['Longitude'], cluster_4['District'], cluster_4['Cluster Labels'], cluster_4['1st Most Common Venue']):
        label = '{} ({}) - {}'.format(dist, cluster, venue1)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_cluster_4 = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(cluster_4, 'red', map_cluster_4)
map_cluster_4

Conclusions:
Cluster 1: It is solely in the 13th district of Paris, also known as 'le quartier chinoise' and we mainly find vienamien/chinese (or Asian) restaurants there.

Cluster 2: It is solely in the 12th district and the most popular venue is a zoo followed by a park.The zoo and the park is actually Zoo/bois de Vincennes and is the largest green cover within Paris.

Cluster 3: It is solely in the 16th district and the most popular venue is a lake. There is a small lake in Bois de Boulogne but it is not so popular. But, perhaps the Eiffel Tower and the riven cruise on the Seine in the 15th which are close by are more popular. The 2nd popular venue is a plaza: there exists several boutiques on the periphery of 15th and 16th arrondissements.

Cluster 4: There are venues from the 7th, 8th, 9th, 14th and 17th districts (French restaurants and hotels are most popular).
7th, 8th and 9th districts are very popular with the tourists while 14th district is preffered by the youngsters.

Summary: Paris being a very touristic city, the results are bound to be more biased towards hotels and French restaurants. A more detailed study is necessary to find popular venues apart from these.